# Relación entre variables

Dos variables pueden estar relacionadas entre si de varias formas. En esta sección vamos a ver como podemos averiguar las relaciones que pueden existir entre dos variables.

Pero antes veamos unos conceptos básicos:

La variable $Y$ tiene una variación lineal respecto a $X$ si:
\\[
Y=\beta·X+a
\\]
Donde $\beta$ y $a$ son constantes fijas que definen la relación entre las dos variables. Pero en cambio, no tendremos una relación lineal en los siguientes casos:
\\[
Y=\beta·X^2+a
\\]
\\[
Y=\beta^X
\\]

# Covarianza

La covarianza es un valor que indica el grado de variación **lineal** conjunta de dos variables aleatorias respecto a sus medias.

Supongamos que queremos comparar dos variables aleatorias X e Y:
* Tendremos alta covarianza (positiva) cuando, para valores altos de X, tengamos mayoritariamente valores altos de Y
* Tendremos baja covarianza (negativa) cuando, para valores altos de X, tengamos mayoritariamente valores bajos de Y
* Tendremos covarianza cercana a 0, para valores altos de X, los valores de Y pueden ser altos o bajos por igual


Su formula es la siguiente:
\\[
cov(X,Y) = \frac{1}{N} \sum _{i=1}^N \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)
\\]

Recordemos la formula de la varianza:
\\[
Var(x) =  \frac{1}{N} \sum _{i=1}^N \left( x_i-\bar{x} \right)^2
\\]

La covarianza de una variable aleatoria consigo misma es igual a la varianza:
\\[
cov(X,X) = Var(X)
\\]

En R la calculamos con la función *cov(x,y)*


In [ ]:
options(repr.plot.height=4,repr.plot.width=6)

n <- 100
x <- rnorm(n,sd=1)
y1 <- 20*x+1+rnorm(n,sd=10)
y2 <- rnorm(n,mean=2, sd=10)
paste("La covarianza de las variables x, y1 es:",round(cov(x,y1),2))
paste("La covarianza de las variables x, y2 es:",round(cov(x,y2),2))
plot(x,y1,col='red')
points(x,y2,col="blue")

En cambio en el siguiente ejemplo la covarianza falla al tratar de encontrar una relación entre dos variables al no ser esta lineal.

In [ ]:
y <- x^2
paste("La covarianza de las variables x, y es:",round(cov(x,y),2))
plot(x,y,col='red')

### Ejemplo para finanzas


En el mercado bursátil, se pone un gran énfasis en reducir el riesgo asumido. Esto generalmente significa que estas acciones no se mueven en la misma dirección. Se suelen elegir acciones que tienen una menor covarianza entre si.
Esto se suele hacer no sobre el valor de la acción en sí, sino sobre la tasa de retorno al final de cada día.

In [ ]:
suppressWarnings(require("quantmod"))
start <- as.Date("2000-01-01")
end <- as.Date("2021-01-01")
TEF<-getSymbols("TEF.MC", src = "yahoo", from = start, to = end,auto.assign = F)
SAN<-getSymbols("SAN.MC", src = "yahoo", from = start, to = end,auto.assign = F)
BBVA<-getSymbols("BBVA.MC", src = "yahoo", from = start, to = end,auto.assign = F)

In [ ]:
candleChart(TEF[1:10,])

In [ ]:
head(TEF)
str(TEF)

In [ ]:
convertToDf<-function(stock){
    valueAdjusted <- stock[,6]
    df <- data.frame(index(valueAdjusted), valueAdjusted, row.names=1:length(valueAdjusted))
    colnames(df) <- c("date",gsub(".Adjusted","",names(valueAdjusted)))
    df
}

df_TEF <- convertToDf(TEF)
df_SAN <- convertToDf(SAN)
df_BBVA <- convertToDf(BBVA)
head(df_TEF)
str(df_SAN)

In [ ]:
df_total <- merge(df_TEF,df_SAN,by="date", all=T)
df_total <- merge(df_total,df_BBVA,by="date", all=T)

seq1 <- 1:(nrow(df_total)-1)
seq2 <- 2:nrow(df_total)

roi <- (df_total[seq2,2:ncol(df_total)]/df_total[seq1,2:ncol(df_total)]-1)*100


head(roi)

In [ ]:
paste("La covarianza entre las tasas de retorno del BBVA y TEF es:",
      cov(roi$BBVA.MC,roi$TEF.MC, use="complete.obs"))
paste("La covarianza entre las tasas de retorno del BBVA y SAN es:",
      cov(roi$BBVA.MC,roi$SAN.MC, use="complete.obs"))

Como cabría esperar existe una gran covarianza entre los bancos, Santander y BBVA, que no se da con Telefónica. Pero la covarianza tiene un problema y es que no está normalizada.

La covarianza entre dos variables puede ser muy alta porque la varianza de una de las variables es muy alta o porque existe una relación alta entre cada una de ellas.

## Matriz de covarianza

La matriz de covarianza muestra la varianza entre $n$ variables en forma de matriz $n \times n$, donde el valor de la celda ij es la covarianza de la secuencia i con la secuencia j, el valor de la diagonal es la varianza de la secuencia correspondiente.

In [ ]:
cov(cbind(x,y1,y2))

La matriz de covarianza nos sirve para de de un vistazo general la relación entre las diferentes variables. Si volvemos al caso anterior de las acciones se puede ver claramente que acciones están más relacionadas entre sí y cuales presentan una mayora varianza.

In [ ]:
cov(roi, use="complete.obs")

# Correlación

La correlación es un valor que indica el grado de variación conjunta y **lineal** de dos variables aleatorias. Es la covarianza normalizada en el rango $[-1,1]$. Es una forma de ignorar la variación de cada una de las variables en si y centrarse únicamente en la relación que existe entre ambas, ya que una covarianza alta puede venir dada también porque una de las variables a estudiar tenga una varianza elevada.

Supongamos que queremos comparar dos variables aleatorias X e Y:
* Correlación cercana a 1, para valores altos de X, tengamos mayoritariamente valores altos de Y
* Correlación cercana a -1, para valores altos de X, tengamos mayoritariamente valores bajos de Y
* Correlación cercana a 0, para valores altos de X, los valores de Y pueden ser altos o bajos por igual


La función de correlación de Pearson es:
\\[
\rho_{X,Y} = corr (X,Y) = \frac{cov(X,Y)}{\sigma_X \sigma_Y} 
\\]


Al igual que con la covarianza podemos calcular una matriz de correlación. Se utiliza para ver de forma sencilla cual es la relación entre varias variables. En una matriz de correlación la diagonal será siempre 1 (la correlación de una variable consigo misma es 1) y el valor de la celda *ij* vendrá dado por la correlación de la variable i con j.


En R la calculamos con la función *cor(x,y)*


In [ ]:
cr1<-cor(x,y1)
cr2<-cor(x,y2)
paste("La correlación de las variables x,y1 es:",round(cr1,2))
paste("La correlación de las variables x,y2 es:",round(cr2,2))

cr<-cor(x,y)
paste("La correlación de las variables x,y es:",round(cr,2))

## Correlación no implica causalidad


https://es.wikipedia.org/wiki/Cum_hoc_ergo_propter_hoc: 

*Cum hoc ergo propter hoc (en latín, "Con esto, por tanto a causa de esto") es una falacia (es decir, un argumento que parece válido, pero que no lo es) que se comete al inferir que dos o más eventos están conectados causalmente porque se dan juntos. La falacia consiste en inferir que existe una relación causal entre dos o más eventos por haberse observado una correlación estadística entre ellos. Esta falacia muchas veces se refuta mediante la frase «correlación no implica causalidad».*

Que dos variables estén correladas no implica que una una es la causa de la otra, es decir, que haya una relación directa entre ambas.

Por ejemplo, existe una correlación de 0.992558 entre la tasa de divorcio en el estado de Maine y el consumo de margarina por habitante:
![](pics/correlacion-margarina-divorcio.png)

Fallos comunes que pueden llevar a buscar una correlación alta:
* Utiliza solo la ventana de datos que te interesa.
* Compara muchas variables entre sí, por puro azar algunas evolucionarán a la par.

Más ejemplos en [tylervigen](http://www.tylervigen.com/spurious-correlations).



Relacionada: https://xkcd.com/552/

Dos variables también pueden estar correladas entre sí, pero la causalidad puede ser debida a una tercera variable oculta que no vemos.
Por ejemplo:

In [ ]:
set.seed(2807)

x<- rnorm(100)
y <- 2*x + 1+ rnorm(100,0,0.1)
paste("La correlación entre x,y es",cor(x,y))


z <- 2*x + 1+ rnorm(100,0,0.1)
paste("La correlación entre x,y es",cor(x,z))


paste("Aunque no existe ninguna relación directa entre z,y existe una alta correlación:", cor(y,z))

#### Correlación entre colesterol y ataques al corazón

Existe una alta correlación en aquellos paises cuyos habitantes toman grasas saturadas (asociadas con un nivel de colesterol más alto) y aquellos cuyos número de enfermedades cardiovasculares es más elevado.

Pero existe un problema en esta correlación, Francia y Finlandia presentan aproximadamente los mismos niveles de ingesta de grasas saturadas, pero los franceses tienen un porcentaje bajo de enfermedades cardiovasculares.

Parece que hay algún otro componente en la dieta que puede estar relacionado con la calidad de la alimentación que es el que realmente influye a la hora de tener o no una enfermedad cardiovascular.


[The French paradox: lessons for other countries](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1768013/)


[Differences in Coronary Mortalit Can Be
Explained by Differences in Cholesterol and
Saturated Fat Intakes in 40 Countries but
Not in France and Finland
A Paradox](https://pdfs.semanticscholar.org/fb57/b6dddc3649702b9e79ae4575ad8a38970d8c.pdf)

### Ejemplo de Matriz de correlación: Mercado continuo

Si se decide invertir en acciones, una política conservadora consiste en diversificar lo máximo posible. Es decir, invertir en empresas que no tengan relaciones. De esta forma si un sector entra en crisis, las empresas de otro sector pueden no verse perjudicadas y salvaremos parte de nuestra inversión.

Un análisis muy sencillo puede ser mediante la matriz de correlación de diferentes empresas. Empresas que presentan un alto nivel de correlación presentan subidas y bajadas muy parecidas. De esta forma, una jugada inteligente podría ser tener un portfolio de acciones con baja correlación.

Veamos un ejemplo con unas pocas empresas y el cambio de moneda Bitcoin-Euro y el Nasdaq

In [ ]:
suppressWarnings(require("quantmod"))
start <- as.Date("2000-01-01")
end <- as.Date("2018-01-01")
AAPL<-getSymbols("AAPL", src = "yahoo", from = start, to = end,auto.assign = F)
BTCEUR<-getSymbols("BTC-EUR", src = "yahoo", from = start, to = end,auto.assign = F)
IXIC<-getSymbols("^IXIC", src = "yahoo", from = start, to = end,auto.assign = F)
TEF<-getSymbols("TEF.MC", src = "yahoo", from = start, to = end,auto.assign = F)
VOD<-getSymbols("VOD", src = "yahoo", from = start, to = end,auto.assign = F)
SAN<-getSymbols("SAN.MC", src = "yahoo", from = start, to = end,auto.assign = F)
BBVA<-getSymbols("BBVA.MC", src = "yahoo", from = start, to = end,auto.assign = F)
TSLA<-getSymbols("TSLA", src = "yahoo", from = start, to = end,auto.assign = F)

In [ ]:
convertToDf<-function(stock){
    valueAdjusted<-stock[,6]
    df<-data.frame(index(valueAdjusted),valueAdjusted,row.names = 1:length(valueAdjusted))
    colnames(df)<-c("date",gsub(".Adjusted","",names(valueAdjusted)))
    df
}
df_IXIC<-convertToDf(IXIC)
df_AAPL<-convertToDf(AAPL)
df_TSLA<-convertToDf(TSLA)
df_TEF<-convertToDf(TEF)
df_VOD<-convertToDf(VOD)
df_SAN<-convertToDf(SAN)
df_BBVA<-convertToDf(BBVA)
df_BTCEUR<-convertToDf(BTCEUR)

head(df_IXIC)

In [ ]:
df_total<-merge(df_IXIC,df_AAPL,by="date",all = T)
head(df_total)

In [ ]:
df_total<-merge(df_total,df_TSLA,by="date",all = T)
df_total<-merge(df_total,df_TEF,by="date",all = T)
df_total<-merge(df_total,df_VOD,by="date",all = T)
df_total<-merge(df_total,df_SAN,by="date",all = T)
df_total<-merge(df_total,df_BBVA,by="date",all = T)
df_total<-merge(df_total,df_BTCEUR,by="date",all = T)


tail(df_total)

In [ ]:
cor(df_total[2:ncol(df_total)],use="pairwise.complete.obs")

Podemos ver como Apple y Tesla están bastante correlados con el Nasdaq, como cabría esperar, al ser este un índice de empresas tecnológicas en USA.

Telefónica y Vodafone se encuentran algo correlados, ambos pertenecen al mismo sector.

Se ve como la correlación entre el banco Santander y el BBVA es alta, esto implica que ambas empresas tienen un comportamiento muy parecido. Según esto daría igual comprar acciones del Santander o del BBVA, las ganancias serían las mismas aproximadamente.

Como cabría esperar el cambio Bitcoin-Euro se encuentra poco correlado con el resto de empresas. Hay una correlación con Apple y TESLA, pero seguramente se deba más a una casualidad, las acciones de Apple y TESLA han subido mucho en los últimos años igual que el Bitcoin. El hecho de que TESLA incorporara BTC en su cartera a mediados de febrero todavía no tiene peso suficiente para alterar la correlación de un año.

### Si la correlación no implica causalidad, ¿qué es entonces la correlación?

Para entender que implicaciones tiene la correlación hay que hacer un análisis causal. Es una rama de la estadística que aunque tiene ya varios años no se encuentra todo lo distribuida que debiera.

El libro de Judea Pearl, [The book of Why](http://bayes.cs.ucla.edu/WHY/) es un gran libro para ayudar a entender realmente la rama de la estadística desde un punto de vista causal.

<img src="http://bayes.cs.ucla.edu/WHY/Pearl-The-Book.jpg" alt="Drawing" style="width: 200px;"/>


# Regresión lineal


Modelo matemático usado para aproximar la relación de dependencia entre una variable dependiente $Y$, la variables independiente $X$ y un término aleatorio $\varepsilon$. Este modelo puede ser expresado como:

\\[
y_i=\beta_0+\beta_1 x_i+\varepsilon_i  \quad \text{para }i=1,\dots,n
\\]

In [ ]:
options(repr.plot.height=5,repr.plot.width=8 , repr.plot.res = 400)

n<-100
x<-rnorm(n,sd=1)
y<- 1+20*x+rnorm(n,mean=1,sd=10)

plot(x,y)
#abline(c(1,20),col="blue")
grid()


Esto significa que se puede expandir de la forma:
\\[
\begin{split}
y_1 &=\beta_0+\beta_1 x_1+\varepsilon_1  \\
y_2 &=\beta_0+\beta_1 x_2+\varepsilon_2  \\
\vdots& \\
y_n&=\beta_0+\beta_1 x_n+\varepsilon_n 
\end{split}
\\]


donde:
* $y_i$: i-esimo valor de la variable dependiente
* $x_i$: i-esimo valor de la variable independiente
* $\beta_0, \beta_1$: parámetros a determinal que dado un conjunto de $x_i$ produce los mejores $y_i$
 * $\beta_0$ : Puede ser llamado sesgo, bias, intercept o término constante. Indica el corte en el eje Y
 * $\beta_1$ : Puede ser llamado pendiente, slope. Indica cuanto aumenta Y por cada incremento de X
* $\varepsilon_i$: error, valor aleatorio.

Podemos escribirlo en forma de matriz:
\\[
\begin{bmatrix}
 y_1 \\ 
 y_2 \\ 
 \vdots \\ 
 y_n 
\end{bmatrix}
=
\begin{bmatrix}
 1 & x_1\\ 
 1 & x_2 \\ 
 \vdots & \vdots \\ 
 1 & x_n 
\end{bmatrix}
\begin{bmatrix}
\beta_0
\\ 
\beta_1
\end{bmatrix}+
\begin{bmatrix}
 \varepsilon_1 \\ 
 \varepsilon_2 \\ 
 \vdots \\ 
 \varepsilon_n
\end{bmatrix}
\\]
Lo que se puede simplificar como:
\\[
Y=X \beta+\varepsilon
\\]




Donde:
* $Y \in \mathbb{R}^{n \times 1}$ 
* $X \in \mathbb{R}^{n \times 2}$ 
* $\beta \in \mathbb{R}^{1 \times 2}$ 
* $\varepsilon \in \mathbb{R}^{n \times 1}$ 




In [ ]:
n<-10
beta<-matrix(c(5,2),nrow = 2)
X<-matrix(c(rep(1,n),1:n),ncol = 2, byrow = F)

print("El valor de la matriz X es:")
X
print("El valor de la Beta es:")
beta

In [ ]:
print("El valor de la matriz X·Beta es:")
X %*% beta

In [ ]:
print("El valor final de la matriz Y es:")
e<-rnorm(n)
Y<-X %*% beta +e
Y

In [ ]:
plot(X[,2],Y,ylim = c(0,30))
abline(beta,col="blue")
grid()

## ¿Cómo calculamos $\beta$?

### Definición de inversa de una matriz

La inversa de una matriz una matriz cuadrada $A$ se escribe $A^-1$ y se define como:
\\[
A^{-1} \cdot A=I=A \cdot A^{-1}
\\]

Donde I es la matriz identidad:
\\[
\begin{bmatrix}
1 & 0 & \dots & 0\\ 
0 & 1 & \dots & 0 \\ 
\vdots &  & \ddots &  \vdots \\ 
0 & 0  & \dots & 1
\end{bmatrix}
\\]



In [ ]:
A<-matrix(c(1,3,5,2,3,4,1,7,8),nrow=3)
print("Tenemos una matriz A:")
A

In [ ]:
print("La inversa de A es:")
iA<-solve(A)
iA

In [ ]:
print("Si multiplicamos una matriz por su inversa obtenemos una matriz identidad")
iA %*% A

Podemos usar la matriz inversa para resolver ecuaciones:
\\[
\begin{split}
u&=Av \\
A^{-1}u&=A^{-1}Av \\
A^{-1}u&=Iv \\
A^{-1}u&=v \\
\end{split}
\\]



In [ ]:
print("Calculamos el vector u como resultado de multiplicar A·v")
v <- c(1,2,3)
u <- A %*% v
u

In [ ]:
print("Comprobamos que multiplicando u por la inversa de A obtenemos v de vuelta:")
iA %*% u

### Pseudoinversa Moore-Penrose

¿Qué ocurre si la matriz que queremos invertir no es cuadrada? Entonces el sistema de ecuaciones no tiene una única solución o simplemente no tiene solución.

Lo que tratamos de encontrar es la menos mala de las soluciones.

Para números reales se define como:
\\[
X^+ = \left( X^{\intercal} X  \right)^{-1} X^{\intercal}
\\]
Cuando:
\\[
X^+X = I
\\]



El problema original consistía en querer calcular $\beta$ de tal forma que se minimice la influencia de $\varepsilon$:
\\[
Y=X \beta+\varepsilon
\\]
En nuestro sistema $\varepsilon$ es desconocido, pero sabemos que tiene media cero y varianza finita. Así que  simplificamos el siguiente sistema de ecuaciones:

\\[
\begin{split}
Y=&X \beta' \\
X^+Y=& X^+X \beta' \\
X^+Y=& \beta' \\
\left( X^{\intercal}X  \right)^{-1} X^{\intercal} Y =& \beta'
\end{split}
\\]

Ahora tenemos, a partir de $X$ e $Y$ una estimación $\beta'$. Podemos entonces, a partir de valores conocidos de $X$ calcular una estimación de $Y'$. 
\\[
Y'=X \beta'
\\]

Evidentemente $Y \neq Y'$, llamaremos **residuo**(error) a la diferencia entre el valor real y el valor estimado : $\varepsilon' = Y-Y'$. 

El método anterior garantiza que el error cuadrático medio sea mínimo.
\\[
{1 \over n} \sum_{i=0}^n{(Y-Y')^2}
\\]

In [ ]:
print("El valor estimado de beta")
est_beta <- solve(t(X) %*% X) %*% t(X) %*% Y
est_beta

In [ ]:
X

In [ ]:
est_Y <- X %*% est_beta
plot(X[,2],Y,col="blue")
points(X[,2],est_Y,col="red")
abline(est_beta,col="red")
abline(c(5,2),col="black")


legend(1,30,legend=c("Y","Y'"),col=c("blue","red"),pch=1)
grid()
t(Y-est_Y)

In [ ]:
paste("El error cuadrático medio es:",mean((Y-est_Y)^2))

### Método en R

En R existe la función lm (linear model) que se utiliza para calcular la dependencia entre variables

In [ ]:
n<-10
beta<-matrix(c(5,2),nrow = 2)
e<-rnorm(n,sd=1)
X<-matrix(c(rep(1,n),1:n),ncol = 2, byrow = F)
Y<-X %*% beta +e

In [ ]:
datos <- data.frame(Y=Y,X=X[,2])
head(datos)

In [ ]:
model <- lm(data=datos, formula= Y ~ X)
summary(model)

In [ ]:
model$coefficients

## Fiabilidad de los coeficientes

Suponiendo que los residuos siguen una distribución gaussiana, se puede calcular la varianza de cada uno de los coeficientes como:
\\[
Var [ \beta' | X ] = s^2 (X^TX)^{-1} = \frac{\varepsilon^2}{n-p} ·  (X^TX)^{-1} = \frac{\sum(Y_i-Y')^2}{n-p} ·  (\sum(X_i-\overline{X}) )^{-1}
\\]

Donde:
* p es el número de grados de liberdad, de coeficientes, en este caso son 2: Intersección y la pendiente.
* n es el número de muestras

La media de los coeficientes vienen dados por el vector $\beta'$. Con la media y la varianza podemos calcular la probabilidad de que el valor de la media estimado esté muy desviado de 0 por puro azar. Eso aparece con el valor Pr(>|t|) en el summary del modelo. Cuanto menos sea este valor más fiable será. Más adelante veremos que significa esta probabilidad en detalle, llamada p-valor.


In [ ]:
vcov_matrix<-as.numeric(t(model$residuals)%*%model$residuals/(length(model$residuals)-2))*solve(t(X)%*%X)
# vcov_matrix <- vcov(model)
# 1/sum((X[,2]-colMeans(X)[2])^2) == solve(t(X)%*%X)[2,2]

paste("p-valor para intersección:",(1-pt(model$coefficients[1]/sqrt(diag(vcov_matrix))[1],8))*2)
paste("p-valor para pendiente:",(1-pt(model$coefficients[2]/sqrt(diag(vcov_matrix))[2],8))*2)

In [ ]:
vcov_matrix

Podemos calcular el margen de error para la intersección conociendo su desviación típica y que sigue una distribución t-student.

Porque para un intervalo de confianza del 95% la formula de los márgenes superior e inferor es:

\\[
inf_{0.95}=Qt(0.975)·\sigma + \bar x \\
sup_{0.95}=Qt(0.025)·\sigma + \bar x
\\]

In [ ]:
library(ggplot2)
options(repr.plot.height=2,repr.plot.width=6)

mydt<-function(x,df,mn,sd){
    dt((x-mn)/sd,df)
}
df<-n-2

ggplot(data=datos, aes(x=X)) +     
    stat_function(fun=mydt,args = list(df = df,mn=model$coefficients[1],sd=sqrt(diag(vcov_matrix))[1]),color="#2222BB")+
    geom_vline(xintercept=qt(0.975,df,lower.tail = F)*sqrt(diag(vcov_matrix))[1]+model$coefficients[1])+
    geom_vline(xintercept=qt(0.025,df,lower.tail = F)*sqrt(diag(vcov_matrix))[1]+model$coefficients[1])+
    xlim(-10,40)

Con R podemos calcular esto automáticamente mediante el comando *confint*:

In [ ]:
cnf_int<-confint(model)
cnf_int

**Opción interval = 'confidence':**

Un **intervalo de confianza** de la predicción es un rango que probablemente contiene el **valor medio de la variable dependiente** dados los valores específicos de las variables independientes. Estos intervalos proporcionan un rango para el promedio de la población. Estos rangos no dicen nada sobre la distribución de los puntos de datos individuales alrededor de la media de la población.

**Opción interval = 'prediction':**

Un **intervalo de predicción** es un rango que probablemente contiene el **valor de la variable dependiente** para una sola observación nueva dados los valores específicos de las variables independientes. Con este tipo de intervalo, estamos prediciendo rangos para observaciones individuales en lugar del valor medio.

In [ ]:
pred_conf<-predict(model,datos,interval = 'confidence')
head(pred_conf)
pred<-predict(model,datos,interval = 'prediction')
head(pred)

In [ ]:
options(repr.plot.height=4,repr.plot.width=6)

est_Y <- X %*% model$coefficients
plot(X[,2],Y,col="blue")
#points(X[,2],est_Y,col="red")
#lines(X[,2],pred[,"fit"],col="red")
#lines(X[,2],pred_conf[,"lwr"],col="black")
#lines(X[,2],pred_conf[,"upr"],col="black")
#lines(X[,2],pred[,"lwr"],col="gray")
#lines(X[,2],pred[,"upr"],col="gray")


#abline(c(5,2),col="green")

## Coeficiente de determinación $R^2$


Proporciona una medida de como de bien nuestra medida sigue al modelo. Se calcula mediante:

\\[
R^2=1-\frac{SS_{res}}{SS_{tot}}=1-\frac{MSE(y,y')}{VAR(y)}
\\]

Donde $SS_{res}$ es la suma del cuadrado de los residuos:
\\[
SS_{res}=\sum_i (y_i-y_i')^2
\\]

y $SS_{tot}$ es proporcional a la varianza de $Y$:

\\[
SS_{tot}=\sum_i (y_i-\bar{y})^2
\\]

Cuanto más cercano a $1$ mejor seguirá la predicción a los datos reales.

Responde a la pregunta, ¿como de mejor es mi modelo respecto a uno que siempre devuelva el valor medio?

In [ ]:
Rsq <- 1-(sum((Y-est_Y)^2))/(sum((Y-mean(Y))^2))
print(paste("El coeficiente de determinación es:",Rsq))

In [ ]:
summary(model)

Nuevo ejemplo:

In [ ]:
options(repr.plot.height=4,repr.plot.width=6)
n<-40
xn<-rnorm(n,sd=1)
yn<-xn*2+rnorm(n,mean=2,sd=10)
datos<-data.frame(y=yn,X=xn)
model=lm(datos, formula=y~X+0)


plot(xn,yn,col="blue")
abline(c(0,model$coefficients),col="red")
summary(model)$r.squared

In [ ]:
summary(model)

In [ ]:
options(repr.plot.height=2,repr.plot.width=6)

vcov_matrix<-as.numeric(t(model$residuals)%*%model$residuals/(length(model$residuals)-2))*solve(t(xn)%*%xn)


ggplot(data=datos, aes(x=X)) +     
    stat_function(fun=mydt,args = list(df = df,mn=model$coefficients[1],sd=sqrt(diag(vcov_matrix))[1]),color="#2222BB")+
    geom_vline(xintercept=qt(0.975,df)*sqrt(diag(vcov_matrix))[1]+model$coefficients[1])+
    geom_vline(xintercept=qt(0.025,df)*sqrt(diag(vcov_matrix))[1]+model$coefficients[1])+
    xlim(-20,20)

In [ ]:
cnf_int<-confint(model)
cnf_int

In [ ]:
options(repr.plot.height=4,repr.plot.width=6)

datos<-datos[order(datos$X),]
pred<-predict(model,datos,interval="confidence")
est_Y <- pred[,"fit"]
plot(xn,yn,col="blue")
points(datos$X,est_Y,col="red")

#lines(datos$X,pred[,"fit"],col="red")
#lines(datos$X,pred[,"lwr"],col="black")
#lines(datos$X,pred[,"upr"],col="black")
#abline(c(0,cnf_int[2]),col="gray")
#abline(c(0,cnf_int[1]),col="gray")



Relacionado: https://xkcd.com/1725/

### Ejemplo: Evolución de la producción de cereales por hectárea en Italia

El siguiente gráfico muestra como ha evolucionado la producción de cereales por hectarea cultivada en italia desde 1960 hasta 2018.

Alrededor del mundo se observa una tendencia muy parecida a partir de los años 60, nuevas especies híbridas de cereales junto con nuevos fertilizantes químicos y pesticidas, aumentarion la producción agrícola en todo el mundo en lo que se acabó llamando la revolución verde. El padre de este movimiento es considerado Norman Borlaug, premio nobel de la paz en 1970.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/ba/Norman_Borlaug.jpg/220px-Norman_Borlaug.jpg)




In [ ]:
library(ggplot2)
yieldKgIt<-read.csv("data/yieldKgIt.csv")

In [ ]:
model <- lm(data=yieldKgIt, formula=yieldCereal~DATE)
summary(model)
yieldKgIt$predicted <- predict(model, yieldKgIt)

ggplot(data=yieldKgIt,aes(x=DATE))+
   geom_point(aes(y=yieldCereal))+
   geom_line(aes(y=predicted),color="blue")+
   theme_bw()+xlab("Año")+ylab("Kg/ha")+ggtitle("Producción de cereales")


Cada año, de media, la producción de cereales aumenta en 67.5kg/ha, sigue una curva lineal con bastante poco error. En la siguiente gráfica se ve los residuos:

In [ ]:
ggplot(data=yieldKgIt,aes(x=DATE,y=yieldCereal-predicted))+   
   geom_line(color="blue")+
   geom_point(color="red")+
   theme_bw()+xlab("Año")+ylab("Kg/ha")+ggtitle("Residuos")


¿Porqué a veces la producción de cereales es inferior o superior con la curva esperada?
¿Podría haber factores meteorológicos involucrados?

No tenemos el histórico de meteorología de toda italia, pero podemos aproximarlo con el histórico de la ciudad de Bolonia.


In [ ]:
residuals<-data.frame(DATE=yieldKgIt$DATE,res=model$residuals)
weather<-read.csv("data/ITE00100550.csv")
wc<-merge(weather[,c("DATE","PRCP")],residuals,by="DATE")
cor(wc$PRCP,wc$res,use="complete.obs")
#ggplot(wc,aes(x=DATE,y=PRCP))+geom_point()

# Regresión lineal múltiple

Hasta ahora habíamos visto como el cambio en una variable puede afectar a otra, pero ¿qué ocurre si son varias las variables que alteran el comportamiento de nuestra variable objetivo? En este caso necesitamos utilizar la regresión lineal múltiple.

Es un modelo matemático usado para aproximar la relación de dependencia entre una variable dependiente $Y$, las variables independientes $X_i$ y un término aleatorio $\varepsilon$. Este modelo puede ser expresado como:

\\[
Y=\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p+\varepsilon = \sum \beta_k X_k+\varepsilon
\\]


donde:
* $Y$: variable dependiente
* $X_1,X_2,\cdots ,X_p$: variables independientes
* $\beta_0, \beta_1,\beta_2,\cdots ,\beta_p$: parámetros a determinal que dado un $X_k$ produce el mejor posible $Y$
* $\varepsilon$: error, valor aleatorio.

Condiciones para aplicarlo:
1. Que la relación entre las variables sea lineal.
2. Que los errores en la medición de $X_k$ sean independientes entre sí.
3. Que los errores tengan varianza constante. (https://es.wikipedia.org/wiki/Heterocedasticidad)
4. Que los errores tengan una media aritmética igual a cero.


Para calcular el valor óptimo de $\beta$ vamos a utilizar un poco de álgebra linea. 
\\[
\begin{split}
y_1 &=\beta_1 x_{11}+\beta_2 x_{12}+\beta_3 x_{13}+\cdots+\beta_p x_{1p}+\varepsilon_1  \\
y_2 &=\beta_1 x_{21}+\beta_2 x_{22}+\beta_3 x_{23}+\cdots+\beta_p x_{2p}+\varepsilon_1  \\
\vdots& \\
y_n &=\beta_1 x_{n1}+\beta_2 x_{n2}+\beta_3 x_{n3}+\cdots+\beta_p x_{np}+\varepsilon_1  \\
\end{split}
\\]

Podemos escribirlo en forma de matriz:
\\[
\begin{bmatrix}
 y_1 \\ 
 y_2 \\ 
 \vdots \\ 
 y_n 
\end{bmatrix}
=
\begin{bmatrix}
 x_{11} & x_{12} & x_{13} & \cdots & x_{1p} \\ 
 x_{21} & x_{22} & x_{23} & \cdots & x_{2p} \\ 
 \vdots & & & \ddots & \vdots \\ 
 x_{n1} & x_{n2} & x_{n3} & \cdots & x_{np}
\end{bmatrix}
\begin{bmatrix}
\beta_1 \\ 
\beta_2 \\
\beta_3 \\
\vdots  \\
\beta_p
\end{bmatrix}+
\begin{bmatrix}
 \varepsilon_1 \\ 
 \varepsilon_2 \\ 
 \vdots \\ 
 \varepsilon_n
\end{bmatrix}
\\]
Lo que se puede simplificar como:
\\[
Y=X \beta+\varepsilon
\\]

Al igual que en el caso anterior podemos estimar $\beta$ con:
\\[
\beta=\left( X^{\intercal}X  \right)^{-1} X^{\intercal} Y
\\]

In [ ]:
n<-10
x1<-rep(1,n)
x2<-seq(2,to=11,length.out = n)
x3<-seq(0.3,to=5,length.out = n)^2
X<-matrix(c(x1,x2,x3),nrow=n,byrow = F)
print("El valor de la matriz X es:")
X
beta<-matrix(c(5,2,14),nrow=3)
print("El valor de la matriz Y es:")
Y<-X %*% beta+rnorm(n)
Y

In [ ]:
print("El valor estimado de beta es:")
est_beta<-solve(t(X) %*% X) %*% t(X) %*% Y
est_beta

print("El valor real de beta es:")
beta

Esto también se puede resolver con la función *lm* de R. Ignoramos el vector *x1* porque la función por defecto ya añade el término.

In [ ]:
lm(Y~x2+x3)

### Ejemplo: Peso de los niños al nacer

Este dataset contien información de bebes recien nacidos y sus padres. Podemos usarlo como regresión para ver cuales son los factores que más afectan al peso del niño.

http://people.reed.edu/~jones/141/BirthWgt.html

Tenemos las siguientes variables que vamos a utilizar:

|Nombre|Variable|
|-|-|
|Birthweight| Peso al nacer (libras)|
|Gestation| Semanas que duró la gestación|
|motherage| Edad de la madre|
|mnocig| Número de cigarros al día fumados por la madre|
|mheight| Altura de la madre (pulgadas)|


In [ ]:
bwt<-read.csv("data/birthweight_reduced.csv")
#str(bwt)


In [ ]:
library(GGally)
options(repr.plot.height=4,repr.plot.width=6)
ggpairs(bwt[,c("Gestation","motherage","mnocig","mheight","Birthweight")],
       lower = list(continuous = wrap("density", alpha = 0.8,size=0.2,color='blue'))
       #lower = list(continuous = wrap("points", alpha = 0.3,size=0.1,color='blue'))
       )

In [ ]:
model<-lm(data=bwt, formula = Birthweight ~ Gestation+motherage+mnocig+mheight)
summary(model)

Los valores que más influencia parecen tener son aquellos que presentan un pvalor (Pr) más bajo. El número de * que hay a la derecha de cada fila indica su grado de confianza.

La variable que más parece influir es la gestación, parece que por cada semana de gestación el bebé gana 0.33062 libras de peso. En cambio por cada cigarro al día que fuma la madre el peso del bebé podría disminuir en 0.02613 libras. La altura de la madre también parece tener cierta incluencia, por cada pulgada más que mida la madre el bebé pesará 0.13329 libras más. En cambio la edad de la madre parece no tener ningún efecto estadístico significativo.

In [ ]:
model<-lm(data=bwt, formula = Birthweight ~ Gestation+mnocig+mheight)
summary(model)

In [ ]:
confint(model)

## Estimación de funciones polinómicas

Podemos estimar funciones polinómicas del tipo:
\\[
y_i=\beta_0+\beta_1·x_i·+\beta_2·x_i^2+\beta_3·x_i^3+\cdots+\beta_p·x_i^p
\\]


Simplemente reescribiendolo como un modelo lineal múltple
\\[
\begin{bmatrix}
 y_1 \\ 
 y_2 \\ 
 \vdots \\ 
 y_n 
\end{bmatrix}
=
\begin{bmatrix}
 1 & x_{1} & x_{1}^2 & x_{1}^3 & \cdots & x_{1}^p \\ 
 1 & x_{2} & x_{2}^2 & x_{2}^3 & \cdots & x_{2}^p \\ 
 \vdots & & & \ddots & \vdots \\ 
 1 & x_{n} & x_{n}^2 & x_{n}^3 & \cdots & x_{n}^p
\end{bmatrix}
\begin{bmatrix}
\beta_0 \\ 
\beta_1 \\
\beta_2 \\
\beta_3 \\
\vdots  \\
\beta_p
\end{bmatrix}+
\begin{bmatrix}
 \varepsilon_1 \\ 
 \varepsilon_2 \\ 
 \vdots \\ 
 \varepsilon_n
\end{bmatrix}
\\]
Lo que se puede simplificar como:
\\[
Y=X \beta+\varepsilon
\\]


In [ ]:
n<-100
x<-seq(-10,10,length.out = n)

x0<-rep(1,n)
x1<-x
x2<-x^2 
x3<-x^3
X<-matrix(c(x0,x1,x2,x3),nrow=n,byrow = F)

beta<-matrix(c(1,1,3,0.5),nrow=4)
Y<-X %*% beta+rnorm(n,sd = 50)

plot(X[,2],Y,pch='*')

est_beta<-solve(t(X) %*% X) %*% t(X) %*% Y
est_Y<-X %*% beta
lines(X[,2],est_Y,col="blue")

print("El valor estimado de beta es:")
est_beta

print("El valor real de beta es:")
beta

In [ ]:
modelPoly3<-lm(Y~x+I(x^2)+I(x^3))
modelPoly3

In [ ]:
summary(modelPoly3)

### Sobreajuste

El sobreajuste u overfitting ocurre en los modelos que tienen un grado polinómico tan alto que empiezan a seguir al ruido, se sobreajustan demasiado a los datos, perdiendo una visión más general de los datos y proporcionando predicciones peores.

En este caso en lugar de estar aprendiendo la relación básica entre dos variables estaríamos aprendiendo los errores, nuestro modelo intenta integrar los errores para dar una respuesta más exacta. El problema es que al coger otra vez variables similares los errores cambian, y lo aprendido con los errores anteriores ya no nos sirve, con lo cual nuestro modelo sería erroneo.

No hay que perderse demasiado en los detalles y las pequeñas variaciones de una variable, la esencia del modelo se basa en encontrar una predicción que funcione para la mayoría de los casos, no para los datos que estamos viendo ahora mismo, sino también para los datos futuros que nos llegarán.

In [ ]:
modelOverfit <- lm(Y~poly(x,27))
summary(modelOverfit)

In [ ]:
plot(X[,2],Y,pch='*')

xp<-seq(-10,10,by=0.1)
yp<-predict(modelOverfit,data.frame(x=xp))
lines(xp,yp,col="blue")
lines(xp,predict(modelPoly3,data.frame(x=xp)),col="red")

In [ ]:
paste("Polinomio grado 27: Rsquared:",summary(modelOverfit)$r.squared)
paste("Polinomio grado 27: Residual standard error",sqrt(sum(summary(modelOverfit)$residuals^2)/(length(summary(modelOverfit)$residuals)-25)))

paste("Polinomio grado 3: Rsquared:",summary(modelPoly3)$r.squared)
paste("Polinomio grado 3: Residual standard error",sqrt(sum(summary(modelPoly3)$residuals^2)/(length(summary(modelPoly3)$residuals)-4)))



### Como encontrar el mejor modelo


Podemos usar el p-valor de los coeficientes o buscar entre todos los modelos el de menor AIC.

El criterio de información de Akaike (AIC) es un estimador de la calidad relativa del modelo que tiene en cuenta su complejidad.

A medida que se aumenta el número de parámetros de entrada o de grados de un polinomio el valor del *error cuadrático medio* va a ser mejor porque el error cuadrático medio disminuye. El AIC penaliza los modelos complejos en favor del los sencillos para evitar el sobreajuste.

\\[
AIC = 2·P+N·log\left(\frac{SS_{res}}{N}\right) = 2·P + N·log \left(\frac{\sum_i (y_i-y_i')^2}{N} \right)
\\]

Donde $P$ es el número de variables que se utilizan en el modelo y $N$ el número de muestras. Cuanto menor sea el vlor del AIC, mejor será el modelo.





In [ ]:
paste("Polinomio grado 27, AIC:",AIC(modelOverfit))
paste("Polinomio grado 3, AIC:",AIC(modelPoly3))

En R existe la función stepAIC que nos encuentra el AIC mínimo tras probar con diferentes combinaciones.

In [ ]:
library(MASS)
datadf<-data.frame(y=Y,x=x)
model<-lm(formula=y~x+I(x^2)+I(x^3)+I(x^4)+I(x^5)+I(x^6)+I(x^7)+I(x^8),datadf)
stepAIC(model,trace=FALSE)

In [ ]:
modelOptim<-#completar
modelOptim
plot(X[,2],Y,pch='*')

xp<-seq(-10,10,by=0.1)
yp<-predict(modelOptim,data.frame(x=xp))
lines(xp,yp,col="blue")

modelPoly3<-lm(Y~x+I(x^2)+I(x^3))
lines(xp,predict(modelPoly3,data.frame(x=xp)),col="red")

legend(-10,800,c("polinomio menor AIC","polinomio grado 3"),pch='-',col=c('blue','red'),y.intersp=2)

In [ ]:
summary(modelOptim)
summary(modelPoly3)

Aunque el error sea menor con el polinomio de grado 24, es mejor utilizar el modelo con un el polinomio de grado 3. Porque el modelo de grado 24 está aprendiendo sobre el ruido.

### Ejemplo: Evolución de la población en España

Volvamos a la distribución por edad de la población española comienzos de 2020. Podemos ver como el número de personas de más de 41 años parece ser representado más o menos por una línea:

In [ ]:
poblacion_raw<-read.csv("data/9663bsc.csv",skip=0,fileEncoding="ISO-8859-1",header=T, sep=';',stringsAsFactor=F)
poblacion_raw$Total <-  as.numeric(gsub('\\.', '', poblacion_raw$Total))

library(dplyr)
library(tidyr)

poblacion <- poblacion_raw %>% pivot_wider(names_from = Sexo, values_from = Total)  %>% 
 mutate(edad=as.numeric(gsub("año(s|)","",Edad.simple))) %>% drop_na() %>% select(-c(Periodo,Edad.simple)) %>%
 rename(Total='Ambos sexos') %>% arrange(edad) %>% mutate(nacimiento=2019-edad) %>% filter(edad>43)

head(poblacion)

In [ ]:
ggplot(data=poblacion,aes(x=edad,y=Total))+geom_line()

model<-lm(data=poblacion,Total~poly(edad,12))
#model<-lm(data=poblacion,Total~edad+I(edad^2)+I(edad^3)+I(edad^4)+I(edad^5)+I(edad^6)+I(edad^7)+I(edad^8)+I(edad^9)+I(edad^11))
summary(model)

In [ ]:
library(MASS)
model_optim<-stepAIC(model,trace=FALSE,direction = "both")
model_optim

In [ ]:
model<-lm(data=poblacion,Total~edad+I(edad^2)+I(edad^3)+I(edad^4)+I(edad^5)+I(edad^6)+I(edad^7)+I(edad^8)+I(edad^9))
summary(model)

In [ ]:
poblacion$predicted <- predict(model,poblacion)
ggplot(data=poblacion,aes(x=edad))+geom_point(aes(y=Total))+
 geom_line(aes(y=predicted),color="blue")


Pero si nos fijamos con detenimiento en el residuo, la diferencia entre el valor real y el esperado, podemos ver como todavía hoy se puede ver que los primero años de la postguerra hubo menos nacimientos.

In [ ]:
ggplot(data=poblacion,aes(x=nacimiento,y=Total-predicted))+geom_point(color="black")+
 geom_line(color="blue")


### Regresión con variables categóricas



Dataset de automóviles y seguros.

https://archive.ics.uci.edu/ml/datasets/Automobile

In [ ]:
autos<-read.csv("./data//imports-85.data", col.names = c("risk", "normalized_losses", 
                                                         "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"), stringsAsFactor=T)
autos<-autos[,c("price","horsepower","length","width","body_style","risk")]
autos$horsepower[autos$horsepower=="?"]<-NA
autos$horsepower<-as.numeric(autos$horsepower)
autos$price<-as.numeric(autos$price)

In [ ]:
levels(autos$body_style)

In [ ]:
aggregate(risk~body_style,autos,mean,na.rm=TRUE)

In [ ]:
table(autos$body_style)

In [ ]:
library(ggplot2)
options(repr.plot.height=4,repr.plot.width=6)
ggplot(autos,aes(y=horsepower,color=body_style))+geom_boxplot()

In [ ]:
model<-(lm(formula=risk ~ horsepower+body_style,data=autos))
summary(model)

In [ ]:
predict(model,data.frame(horsepower=20,body_style='hardtop'))-
predict(model,data.frame(horsepower=20,body_style='convertible'))

# Clasificación - Regresión logística

Si lo que tenemos son dos grupos y queremos realizar una clasificación, tenemos que realizar ciertas modificaciones a la regresión lineal.




La fórmula de la regresión lineal es:
\\[
\hat{Y}=\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p = \sum \beta_k X_k
\\]

Podemos tratar de asignar una probabilidad 

Pero hay un problema porque esta regresión va entre 


Por ejemplo: Trabajamos en un maderera y queremos saber si un tronco es de un abedul o de un abeto basándonos únicamente en el diametro y la longitud del tronco.



In [ ]:
library(ggplot2)
options(repr.plot.height=4,repr.plot.width=6)

arboles<-read.csv("data/arboles.csv")
ggplot(arboles,aes(x=diametro,y=longitud,color=especie))+geom_point(size=0.3)

Podemos hacer una clasificación con un modelo lineal donde creamos una nueva columna *especie.n* y le asignamos:
* abeto  = 1
* abedul = -1

Utilizamos un modelo lineal tal que:
\\[
  especie = \beta_0+\beta_1·diametro+\beta_2·longitud
\\]

Entonces:
* si especie >= 0 es un abeto
* si especie <  0es un abedul

La recta que marcará el umbral será:
\\[
\begin{split}    
  0 &= \beta_0+\beta_1·diametro+\beta_2·longitud \\
  -\beta_2 longitud &= \beta_0+\beta_1·diametro\\
  longitud &= \frac{-\beta_0}{\beta_2}-\frac{\beta_1}{\beta_2}·diametro
\end{split}  
\\]



In [ ]:
arboles$especie.n[arboles$especie=="abeto"] <-   1
arboles$especie.n[arboles$especie=="abedul"]<-  -1

itrain<-sample(1:nrow(arboles),round(nrow(arboles)*0.7))
arboles.train<-arboles[itrain,]
arboles.test<-arboles[-itrain,]


modellm<-lm(data=arboles.train,formula=especie.n~diametro+longitud)
beta<-modellm$coefficients

ggplot(arboles.train,aes(x=diametro,y=longitud,color=especie))+geom_point(size=0.3)+
 geom_abline(intercept = -beta[1]/beta[3],slope = -beta[2]/beta[3], color="red" )

Pero esto no es del todo correcto porque los datos **NO** siguen una distribución gaussiana. 
Cada iteración, cada línea del dataframe, cada árbol medido, podría seguir una distribución de bernoulli.
Es un abeto con probabilidad p o es un abedul con probabilidad (1-p). Esto se puede generalizar con una distribución binomial que indique la probabilidad de que n troncos sean abedules.


Para pasar del dominio de números reales $(-\infty,\infty)$ al de probabilidades $[0,1]$ a vamos a utilizar la **función logística**:
\\[
h(x) = \frac{1}{1+e^{-x}}
\\]
Su inversa se conoce como la función **logit**:
\\[
h^{-1}(p) = log \left( \frac{p}{1-p} \right)
\\]

In [ ]:
x<-seq(-10,10,length.out = 100)
y<-1/(1+exp(-x))
plot(x,y,t="l")

Es decir, tenemos una probabilidad, su valor está en el rango $[0,1]$:
\\[
    p = \frac{1}{1-e^{-\hat{Y}}}= \frac{1}{1-e^{-(\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p)}}   
\\]

Definimos la razón de monomios (Odds ratio) como el cociente entre dos probabilidades, su valor está en el rango $[0,\infty]$:
\\[
 Odds = \frac{p}{1-p}=\frac{\frac{1}{1-e^{-(\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p)}}}{\frac{e^{-(\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p)}}{1-e^{-(\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p)}}}=e^{(\beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p)}
\\]

Si aplicamos el logaritmo a la razón de monomios tenemos un valor que está en el rango $[-\infty,\infty]$:
\\[
 log(Odds) = \beta_1 X_1+\beta_2 X_2+\cdots +\beta_p X_p
\\]



Podemos repetir el ejercicio de antes poniendo el umbral en 0 igualmente pero con regresión logística:

In [ ]:
model<-glm(data=arboles.train,formula=especie~diametro+longitud,family=binomial(link='logit'))
betalg<-model$coefficients

ggplot(arboles.train,aes(x=diametro,y=longitud,color=especie))+geom_point(size=0.3)+
 geom_abline(intercept = -beta[1]/beta[3],slope = -beta[2]/beta[3], color="red" )    +
 geom_abline(intercept = -betalg[1]/betalg[3],slope = -betalg[2]/betalg[3], color="blue" )    

### Ejemplo: Peso de los niños al nacer

Este dataset contien información de bebes recien nacidos y sus padres. 
Podemos usarlo como clasificación para ver cuales son los factores que más afectan al niño en función de si la madre es o no fumadora.

Tenemos las siguientes variables que vamos a utilizar:

|Nombre|Variable|
|-|-|
|Birthweight| Peso al nacer (libras)|
|Gestation| Semanas que duró la gestación|
|motherage| Edad de la madre|
|smoker| Madre fumadora 0/1|


In [ ]:
bwt<-read.csv("data/birthweight_reduced.csv")
bwt$smoker <- factor(bwt$smoker,labels=c("No","Yes"))
str(bwt)

In [ ]:
model <- glm(data=bwt, formula = smoker~Birthweight + Gestation, family=binomial())
summary(model)

In [ ]:
betalg <- model$coefficients
ggplot(data=bwt, aes(x=Birthweight ,y=Gestation, color=smoker))+geom_point()+
 geom_abline(intercept=-betalg[1]/betalg[3], slope=-betalg[2]/betalg[3],color="blue")

Vemos que el valor que más influye en si la madre es fumadora o no es en el peso del niño al nacer. No parece que afecte mucho al tiempo de gestación.

Podemos calcular la probabilidad de que la madre sea fumadora mirando el peso del niño y los coeficientes de la regresión logística:
\\[
    p = \frac{1}{1-e^{-\hat{Y}}}= \frac{1}{1-e^{-(\beta_0+\beta_1 · peso)}}   
\\]


In [ ]:
model <- glm(data=bwt, formula = smoker~Birthweight , family=binomial())
model$coefficients

peso<-seq(4,10,length.out=100)
p <- 1/(1+exp(-(model$coefficients[1]+peso*model$coefficients[2])))
plot(peso,p,t="l",xlab="Peso del niño")

In [ ]:
pesoLibras <- 5
pesoKg <- pesoLibras*0.453592
ods <- predict(model,data.frame(Birthweight=pesoLibras))
print(paste("Para un peso de",pesoKg,"Kg es",exp(ods),
            "veces más probable que la madre sea fumadora"))
print(paste("Para un peso de",pesoKg,
            "Kg la probabilidad de que la madre sea fumadora es de",exp(ods)/(1+exp(ods))))


Otra forma de verlo es decir que cada libra que pesa el niño, la razón de probabilidades  de que la madre se fumadora respecto a la que no lo es:
\\[
log(\frac{p}{1-p})=\beta_0+\beta_1 · peso
\\]
\\[
\frac{p}{1-p}=e^{\beta_0}·e^{\beta_1 · peso}
\\]
Por cada unidad que aumenta el peso, el la razón de probabilidades aumenta $e^{\beta_1}$

In [ ]:
peso1 <- 9
peso2 <- 10

p1 <- 1/(1+exp(-(model$coefficients[1]+peso1*model$coefficients[2])))
p2 <- 1/(1+exp(-(model$coefficients[1]+peso2*model$coefficients[2])))

(p2/(1-p2))/(p1/(1-p1))

In [ ]:
exp(model$coefficients[2])

### Censo

Dataset de: https://archive.ics.uci.edu/ml/datasets/adult

* age: continuous. 
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
* fnlwgt: continuous. 
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 
* education-num: continuous. 
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. 
* sex: Female, Male. 
* capital-gain: continuous. 
* capital-loss: continuous. 
* hours-per-week: continuous. 
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [ ]:
adult<-read.csv("data/adult.data.txt",
                col.names=c("age","workclass","fnlwgt","education","education-num","marital-status",
                           "occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
                           "native-country","50k"), stringsAsFactor=T)

str(adult)

In [ ]:
levels(adult$education)
numlevels<-length(levels(adult$education))
adult$education<-factor(adult$education,levels(adult$education)[c(4,5,6,7,1,2,3,8:numlevels)])

In [ ]:
levels(adult$education)


In [ ]:
model <- glm(data=adult, formula=X50k ~ age+education+sex, family = binomial())
model

In [ ]:
paste("Un hombre tiene ",exp(model$coefficients["sex Male"]),"veces más posibilidades de ganar más de 50k$ que una mujer")


In [ ]:
paste("Cada año que pasa hay ",exp(model$coefficients["age"]),"veces más posibilidades de ganar más de 50k$")


In [ ]:
paste("Una persona con Master tiene ",exp(model$coefficients["education Masters"]),"veces más posibilidades de ganar más de 50k$ que alguien con solo 1st-4th")

In [ ]:
adult_master<-subset(adult,education==" Masters")
model <- glm(data=adult_master, formula=X50k ~ age+sex, family = binomial())
summary(model)

In [ ]:
model <- glm(data=adult_master, formula=X50k ~ age*sex, family = binomial())
summary(model)

In [ ]:
confint(model)

## Dataset Anscombe

Este dataset nos muestra que no nos podemos fiar simplemente por los estimadores de las variables, la representación gráfica nos proporciona una visión más completa.

Existen datasets similares como el datasaurio:
https://www.autodesk.com/research/publications/same-stats-different-graphs

In [ ]:
library(datasets)
anscombe

In [ ]:
mean(anscombe$y1)
mean(anscombe$y2)
mean(anscombe$y3)
mean(anscombe$y4)

In [ ]:
var(anscombe$y1)
var(anscombe$y2)
var(anscombe$y3)
var(anscombe$y4)

In [ ]:
library(ggplot2)
options(repr.plot.height=3,repr.plot.width=4 , repr.plot.res = 400)

ggplot(anscombe)+
 geom_point(aes(x=x1,y=y1),color="red")+
 geom_point(aes(x=x2,y=y2),color="green")+
 geom_point(aes(x=x3,y=y3),color="blue")+
 geom_point(aes(x=x4,y=y4),color="black")

In [ ]:
library(grid)
library(gridExtra)
options(repr.plot.height=4,repr.plot.width=8 , repr.plot.res = 200)

g1<-ggplot(anscombe,aes(x=x1,y=y1))+geom_point(color="red")+geom_smooth(method="lm",color="grey",se=F)
g2<-ggplot(anscombe,aes(x=x2,y=y2))+geom_point(color="blue")+geom_smooth(method="lm",color="grey",se=F)
g3<-ggplot(anscombe,aes(x=x3,y=y3))+geom_point(color="blue")+geom_smooth(method="lm",color="grey",se=F)
g4<-ggplot(anscombe,aes(x=x4,y=y4))+geom_point(color="blue")+geom_smooth(method="lm",color="grey",se=F)
 

grid.arrange(g1, g2, g3, g3,nrow = 2,top = textGrob("Anscombe dataset",gp=gpar(fontsize=20,font=3)))

# Distribución conjunta

La distribución conjunta de dos variables A y B se refiere a la distribución de probabilidad de la intersección de ambos eventos.
Se representa como P(B,A) o  P(B $\cap$ A)

## Variables discretas


AVISO: ESTOS DATOS SON PURAMENTE INVENTADOS

Imagina que tenemos una tabla de contingencia que presenta el número de personas que les gusta Star Trek o StarWars función del sexo:

In [ ]:
tbl<-as.table(matrix(c(122,58,214,120),dimnames = list(c("Star Trek","Star Wars"),c("Mujeres","Hombres")),nrow=2))
tbl

**Probabilidad marginal**

En esta población, ¿Cual es la probabilidad de que a alguien le guste Star Wars?
\\[
P(\text{Star Wars})=\frac{58+120}{122+214+58+120}=0.34
\\]

In [ ]:
print("La probabilidad de que a alguien le gusta Star Trek o Star Wars es:")
rowSums(tbl)/sum(tbl)

**Probabilidad conjunta**

¿Cual es la probabilidad de que a un hombre le guste starwars?
\\[
P(\text{Hombre,Star Wars})=\frac{120}{122+214+58+120}=0.233
\\]

In [ ]:
print("La matriz de la probabilidad conjunta es:")
prop.table(tbl)

**Probabilidad condicional**

Suponiendo que nos fijemos solamente en los hombres, ¿cual es la probabilidad de que Star Wars sea la película más interesante?

\\[
P(\text{Star Wars|Hombre})=\frac{120}{214+120}=0.359
\\]


In [ ]:
print("La matriz condicional para mujeres y hombres:")
prop.table(tbl,margin = 2)

Suponiendo que nos fijemos solamente en los fans de Star Wars, ¿cual es la probabilidad de que el fan sea hombre?

\\[
P(\text{Hombre|Star Wars})=\frac{120}{120+58}=0.674
\\]

In [ ]:
print("La matriz condicional para Star Trek y Star Wars :")
prop.table(tbl,margin = 1)

**Propiedades**
\\[
\begin{split}
P(\text{Star Wars,Hombre})&=P(\text{Hombre,Star Wars}) \\
P(\text{Hombre,Star Wars})&=P(\text{Hombre|Star Wars})·P(\text{Star Wars}) \\
P(\text{Star Wars,Hombre})&=P(\text{Star Wars|Hombre})·P(\text{Hombre})
\end{split}
\\]

## Variables continuas

### Dos variables independientes

En este caso: 
P(B,A)=P(B)·P(A)

Es fácil verlo porque su matriz de covarianza se parece a una matriz diagonal.

In [ ]:
N<-10000
A<-rnorm(N,mean=0,sd=0.5)
B<-rnorm(N,mean=0,sd=1)
df<-data.frame(A,B)

print(paste("Matriz de covarianza:"))
cov(df)

library(ggplot2)
ggplot(df, aes(x=A, y=B))+geom_density2d(aes(colour=..level..)) + 
  stat_density2d(aes(fill=..level..,alpha=..level..),geom='polygon',colour='black') + 
  scale_fill_continuous(low="green",high="red") +    
  geom_point(alpha=0.2,size=0.1)+
  theme_bw()+theme(legend.position="none")+xlim(c(-5,5))+ylim(c(-5,5))

Como son independientes, la probabilidad condicional de B respecto a A es igual a la probabilidad de B

P(B|A)=P(B)

In [ ]:
dfPartialProbs<-data.frame(B=B,var="P(B)")
dfPartialProbs<-rbind(dfPartialProbs,data.frame(B=df$B[df$A>-0.5 & df$A<0.5],var="P(B|-0.5<A<0.5)"))


ggplot(dfPartialProbs, aes(x=B,color=var))+geom_density()+
  theme_bw()

### Dos variables dependientes

En este caso: 
P(B,A)=P(B|A)·P(A)=P(A,B)=P(A|B)·P(B)

En este caso la matriz de covarianza no es diagonal.

In [ ]:
N<-10000
A<-rnorm(N,mean=0,sd=2)
B<-A+rnorm(N,mean=0,sd=2)
df<-data.frame(A,B)

print(paste("Matriz de covarianza:"))
cov(df)

library(ggplot2)
ggplot(df, aes(x=A, y=B))+geom_density2d(aes(colour=..level..)) + 
  stat_density2d(aes(fill=..level..,alpha=..level..),geom='polygon',colour='black') + 
  scale_fill_continuous(low="green",high="red") +    
  geom_point(alpha=0.1,size=0.1)+
  theme_bw()+theme(legend.position="none")+xlim(c(-10,10))+ylim(c(-10,10))

Como son dependientes, la probabilidad condicional de B respecto a A es diferente a la probabilidad de B
\\[
P(B|A) \neq P(B)
\\]

In [ ]:
dfPartialProbs<-data.frame(B=B,var="P(B)")
dfPartialProbs<-rbind(dfPartialProbs,data.frame(B=df$B[df$A>-0.5 & df$A<0.5],var="P(B|-0.5<A<0.5)"))

dfPartialProbs<-rbind(dfPartialProbs,data.frame(B=df$B[df$A>3 & df$A<4],var="P(B|3<A<4)"))


ggplot(dfPartialProbs, aes(x=B,color=var))+geom_density()+
  theme_bw()

# Teorema de Bayes


El teorema de Bayes describe la probabilidad de un evento basado en conocimientos previos que puede estar relacionado con el evento. Su fórmula es bastente simple:
\\[
P(A|B)=\frac{P(B|A)·P(A)}{P(B)}=\frac{P(B,A)}{P(B)}
\\]
donde A y B son nuestras variables aleatorias y $P(B) \neq 0$:
* P(A|B) es la probabilidad condicional del evento A, sabiendo que ha ocurrido B
* P(B|A) es la probabilidad condicional del evento B, sabiendo que ha ocurrido A
* P(A) y P(B) es la probabilidad de observar A y B de forma independiente. Es conocido como la probabilidad **marginal**.
* P(B,A) o  P(B $\cap$ A): es la probabilidad de que ambos eventos A y B ocurran a la vez. 
 * Si son sucesos independientes P(B,A)=P(B)·P(A)
 * Si son sucesos dependientes P(B,A)=P(B|A)·P(A)


![]( ./pics/bayes1.jpg )

En la imagen anterior podemos ver dentro del cuadrado verde disparos rojos y blancos sobre un barco.
En total hay 35 disparos, 20 rojos y 15 blancos.

La probabilidad de elegir un disparo rojo al azar es:
\\[
\begin{split}
P(rojo)&=\frac{20}{35}=\frac{4}{7} \approx  0.571 \\
P(blanco)&=\frac{15}{35}=\frac{3}{7} \approx  0.428
\end{split}
\\]

La probabilidad de, en ese cuadrado, acertar al barco es:
\\[
P(barco)=\frac{4}{35} \approx  0.114
\\]

Si sabemos que se ha realizado un disparo **blanco**, ¿cuál es la probabilidad de que haya dado al **barco**?
\\[
P(barco|blanco)=\frac{3}{15} = 0.3
\\]

Si sabemos que se ha realizado un disparo **rojo**, ¿cuál es la probabilidad de que haya dado al **barco**?
\\[
P(barco|rojo)=\frac{1}{20} = 0.05
\\]

#### Apliquemos Bayes

El **barco** ha sido alcanzado por un disparo ¿Cuál es la probabilidad de que ese disparo sea **rojo**? 

Nos están preguntando por $P(rojo|barco)$. Conocemos P(barco|rojo), P(barco) y P(rojo), así que por el teorema de Bayes:

\\[
P(rojo|barco)=\frac{P(barco|rojo)·P(rojo)}{P(barco)} =\frac{0.05 ·\frac{20}{35} }{\frac{4}{35}}=\frac{1}{4}=0.25
\\]



El **barco** ha sido alcanzado por un disparo ¿Cuál es la probabilidad de que ese disparo sea **blanco**? 

Nos están preguntando por $P(blanco|barco)$. Conocemos P(barco|blanco), P(barco) y P(blanco), así que por el teorema de Bayes:

\\[
P(rojo|barco)=\frac{P(barco|blanco)·P(blanco)}{P(barco)} =\frac{\frac{3}{15} ·\frac{15}{35} }{\frac{4}{35}}=\frac{3}{4}=0.75
\\]


#### Probabilidad conjunta

¿Cual es la probabilidad de que un disparo de al **barco** y además sea **rojo**?

\\[
\begin{split}
P(barco,rojo)&=P(barco|rojo)·P(rojo)=\frac{1}{20}·\frac{20}{35}=\frac{1}{35} \\
P(barco,rojo)&=P(rojo|barco)·P(barco)=\frac{1}{4}·\frac{4}{35}=\frac{1}{35}
\end{split}
\\]


## Arboles bayesianos

Estamos en el año 2025, un terrible virus ha escapado de un laboratorio de máxima seguridad y está infectando a gente alrededor de todo el mundo convirtiendolos en zombies admiradores de [Nyan Cat](https://www.youtube.com/watch?v=QH2-TGUlwu4) aunque luego en apariencia llevan una vida normal. Se calcula que en este momento un **5% de la población** mundial está infectada.

Afortunadamente existe un test al que se puede someter a un individuo para ver si está infectado o no. Pero el test no es 100% fiable, nunca se tiene tanta seguridad. Si la persona está infectada el test acertará un 99% de las veces, en cambio si no está infectada el test acertará un 98% de las veces. Esto genera la siguiente **matriz de confusión**:

|. |Infectado|Sano|
|-|-|-|
|**Test=1**| 99%| 2% |
|**Test=0**| 1%| 98% |


Si elegimos una persona al azar y nuestro test dice que está infectado, ¿que posibilidades hay de que realmente esté infectado?
Queremos saber P(Zombie=1|Test=1)

Utilizando Bayes:
\\[
P(Zombie=1|Test=1)=\frac{P(Test=1|Zombie=1)·P(Zombie=1)}{P(Test=1)}
\\]
Pero desconocemos $P(Test=1)$, aunque podemos obtenerlo mediante:
\\[
\begin{split}
P(Test=1)&=P(Test=1|Zombie=1)·P(Zombie=1)+P(Test=1|Zombie=0)·P(Zombie=0) \\
P(Test=1)&=P(Test=1,Zombie=1)+P(Test=1,Zombie=0)
\end{split}
\\]
Es decir:
\\[
\begin{split}
P(Z=1|T=1)=\frac{P(T=1|Z=1)·P(Z=1)}{P(T=1|Z=1)·P(Z=1)+P(T=1|Z=0)·P(Z=0)} \\
P(Z=1|T=1)=\frac{0.99·0.05}{0.99·0.05+0.02·0.95}=0.7226 \\
\end{split}
\\]


Esta misma información se puede representar con un grafo acíclico:

![]( pics/bayestree1.png )

## Análisis Causal y análisis contrafactual

El análisis causal nos permite ver la relación causa efecto que hay entre diferentes variables.

Veamos un ejemplo sacado del libro "The book of why". Imaginemos que la vacuna de la viruela que se aplica al 99% de la población.

De los vacunados, un 1% desarrolla una reacción y de ese grupo un 1% fallece. Pero ninguno coge viruela.

De los no vacunados, un 2% coge la enfermedad, y de esos enfermos un 20% fallece. 

El diagrama causal sería el siguiente:

![](pics/causal_viruela.jpg)

Si pintamos cada una de esas probabilidades. La probabilidad de sufrir una reacción (R) o la enfermedad (E) en función de la vacuna (V):
\\[
P(R|V)=0.01 \\
P(R|\neg V)=0 \\
P(E|V)=0 \\
P(E|\neg V)=0.02
\\]

Las probabilidades de fallecimiento (M) en función de la reacción (R) o la enfermedad (E) son:
\\[
P(M|R)=0.01 \\
P(M|E)=0.2
\\]

Si partimos de una población de 1 millón de personas, habrá 990.000 vacunados y 10.000 sin vacunar. De todos los vacunados fallecerán 99 personas. De los no vacunados fallecerán 40 personas.

**En esta población muere más gente por las vacunas que por la enfermedad**.

El análisis contrafactual trata de responder a la pregunta, si no se hubiera aplicado la vacuna. ¿Cuanta gente habría fallecido?

La probabilidad de fallecer si te pones la vacuna es:

\\[
P(M|V) = P(M|R)·P(R|V) = 0.01·0.01 = 10^{-4}
\\]

La probabilidad de fallecer si NO te pones la vacuna es:
\\[
P(M|\neg V) = P(M|E)·P(E|\neg V) = 0.02·0.2 = 0.004
\\]



Si el millón de personas no se hubiera vacunado número de fallecidos sería de $10^6·0.004 = 4000$ personas. 

La vacuna ha salvado la vida de $4000-99-40=3861$ personas.


## El problema de Monty Hall

Este problema recibe su nombre tras el programa de televisión "Let's Make a Deal" presentado por Monty Hall en la década de 1970. El enunciado más famoso del problema, extraído de Parade Magazine en 1990:

*Supón que estás en un concurso, y se te ofrece escoger entre tres puertas: detrás de una de ellas hay un coche, y detrás de las otras, cabras. Escoges una puerta, digamos la nº1, y el presentador, que sabe lo que hay detrás de las puertas, abre otra, digamos la nº3, que contiene una cabra. Entonces te pregunta: "¿No prefieres escoger la nº2?". ¿Es mejor para ti cambiar tu elección?*


![]( pics/montyhall.png )


Tenemos tres hipótesis:
* $H_1$: El coche está tras la puerta 1
* $H_2$: El coche está tras la puerta 2
* $H_3$: El coche está tras la puerta 3

Evidentemente, como no tenemos ningún tipo de información adicional:
\\[
P(H_1)=P(H_2)=P(H_3)=\frac{1}{3}
\\]

Nuestra evidencia vendrá de Monty abriendo una puerta, evidentemente sabemos que:
* Monty siempre abrirá una puerta de las 2 que no hayamos seleccionado.
* Monty nunca abrirá la puerta que tenga el coche.


Nosotros estamos interesados en la probabilidad que el coche esté detrás de una puerta utilizando nuestro conocimiento.

Imaginemos que el **concursante selecciona la puerta 1**, a continuación **Monty selecciona la puerta 3**. Las siguientes **probabilidades condicionales** ocurren:
* Condición, el coche está tras la puerta 1: Puede abrir igualmente la puerta 2 o 3
\\[
P(M_3|H_1)=\frac{1}{2}=P(M_2|H_1)
\\]
* Condición, el coche está tras la puerta 2: Puede abrir solo la puerta 3:
\\[
P(M_3|H_2)=1
\\]
* Condición, el coche está tras la puerta 3: No podría abrir esa puerta:
\\[
P(M_3|H_3)=0
\\]

Desde nuestro punto de vista, la probabilidad marginal de que Monty elija la puerta 2 o 3 es la misma, porque no sabemos donde está el coche. Solo sabemos que nunca eligirá la puerta 1 porque es la que seleccionó el concursante:
\\[
P(M_2)=P(M_3)=\frac{1}{2}
\\]


Ahora hay dos opciones, cambiar a la puerta 2 o mantenerse en la puerta 1.
* Se mantiene en puerta 1, sabiendo que la puerta 3 no tiene premio:
\\[
P(H_1|M_3)=\frac{P(M_3|H_1)·P(H_1)}{P(M_3)}=\frac{\frac{1}{2}·\frac{1}{3}}{\frac{1}{2}}=\frac{1}{3}
\\]
* Se cambia a la puerta 2, sabiendo que la puerta 3 no tiene premio:
\\[
P(H_2|M_3)=\frac{P(M_3|H_2)·P(H_2)}{P(M_3)}=\frac{1·\frac{1}{3}}{\frac{1}{2}}=\frac{2}{3}
\\]

Así pues se ve que la probabilidad de cambiar de puerta es 2/3, mientras que la probabilidad de mantenerse con la original es 1/3.


In [ ]:
N<-10000
puerta_con_premio<-floor(runif(N,min=1,max=4))

seleccion_inicial_del_concursante<-floor(runif(N,min=1,max=4))
seleccion_cambiada_del_concursante<-rep(NA,N)

for (i in 1:N){
    
    # Monty selecciona una puerta
    if (puerta_con_premio[i]==1){
        if (puerta_con_premio[i]==seleccion_inicial_del_concursante[i]){
            seleccion_monty<-ifelse(runif(1)>0.5,2,3)
        }else{
            seleccion_monty<-ifelse(seleccion_inicial_del_concursante[i]==3,2,3)
        }
    }else if (puerta_con_premio[i]==2){
        if (puerta_con_premio[i]==seleccion_inicial_del_concursante[i]){
            seleccion_monty<-ifelse(runif(1)>0.5,1,3)
        }else{
            seleccion_monty<-ifelse(seleccion_inicial_del_concursante[i]==3,1,3)
        }        
    }else if (puerta_con_premio[i]==3){
        if (puerta_con_premio[i]==seleccion_inicial_del_concursante[i]){
            seleccion_monty<-ifelse(runif(1)>0.5,1,2)
        }else{
            seleccion_monty<-ifelse(seleccion_inicial_del_concursante[i]==1,2,1)
        }
    }
    
    # El concursante selecciona otra puerta
    puertas_prohibidas<-c(seleccion_monty,seleccion_inicial_del_concursante[i])
    seleccion_cambiada_del_concursante[i]<-which(is.na(match(1:3,puertas_prohibidas)))    
}

p1<-mean(seleccion_inicial_del_concursante==puerta_con_premio)
print(paste0("Si el concursante nunca cambia de puerta la probabilidad de acertar es:",p1))
p2<-mean(seleccion_cambiada_del_concursante==puerta_con_premio)
print(paste0("Si el concursante siempre cambia de puerta la probabilidad de acertar es:",p2))